# **Predição de Default:** 


## **1. Introdução**

Este notebook contém a função de predição do modelo treinado de default de clientes.

A função recebe um dicionário contendo informações sobre um cliente e retorna o valor da predição de default (1 = se terá default ou 0 = será ausente). 


## **2. Modelo Utilizado**

O modelo de Machine Learning utilizado para essa predição foi o **XGBoost Classifier**, ajustado e treinado utilizando **RandomizedSearchCV** para otimização de hiperparâmetros.

## **3. Variáveis de entrada**

* Para garantir uma predição adequada, os seguintes atributos são obrigatórios na entrada da função de predição:

    -  `default_3months`: Indica se o cliente teve default nos últimos 3 meses (1 = Sim, 0 = Não)
    -  `ioi_36months`: Intervalo médio entre pedidos (em dias) nos últimos 36 meses
    -  `ioi_3months`: Intervalo médio entre pedidos (em dias) nos últimos 3 meses
    -  `valor_por_vencer`: Total em pagamentos a vencer do cliente B2B, em Reais
    -  `valor_vencido`: Total em pagamentos vencidos do cliente B2B, em Reais
    -  `valor_quitado`: Total (em Reais) pago no histórico de compras do cliente B2B 
    -  `quant_protestos` : Quantidade de protestos de títulos de pagamento apresentados no Serasa
    -  `valor_protestos` : Valor total (em Reais) dos protestos de títulos de pagamento apresentados no Serasa
    -  `quant_acao_judicial` : Quantidade de ações judiciais apresentadas pelo Serasa
    -  `acao_judicial_valor` : Valor total das ações judiciais (Serasa) 
    -  `dividas_vencidas_valor` : Valor total de dívidas vencidas (Serasa)
    -  `dividas_vencidas_qtd` : Quantidade total de dívidas vencidas (Serasa)
    -  `falencia_concordata_qtd` : Quantidade de concordatas (Serasa)
    -  `tipo_sociedade` : Tipo de sociedade do cliente B2B 
    -  `opcao_tributaria` : Opção tributária do cliente B2B
    -  `atividade_principal` : Atividade principal do cliente B2B
    -  `forma_pagamento` : Forma de pagamento combinada para o pedido
    -  `valor_total_pedido` : Valor total (em Reais) do pedido em questão
    -  `month` : Mês do pedido

*  Se alguma dessas variáveis estiver ausente, a função irá retornar um erro indicando a falta do campo.


## **4. Carregamento do Modelo**

*  O modelo está disponível como um pipeline completo na pasta **model** do presente repositório e contém as seguintes etapas:
    *  Pré-processamento (Feature Engineering, transformação dos dados)
    *  Modelo Treinado (XGBoost)

O modelo pode ser carregado diretamente dentro da função *prever_default()*: 

ex: 

```python
pipeline_caminho_pasta = "../model/pipeline_predicao_default.pkl"
pipeline = joblib.load(pipeline_caminho)
```
# **5. Implementação da Função de Predição**

A função *prever_default()* recebe um **dicionário com os dados do cliente**, realiza o pré-processamento e faz a previsão usando o modelo treinado. 

Importando bibliotecas

In [25]:
# importando bibliotecas 

import sys
sys.path.append("../src")
from custom_transformers import FeatureEngineeringTransformer
import pandas as pd 
import joblib

import warnings
# Ignorar todos os warnings
warnings.filterwarnings("ignore")

Função de Predição:

In [20]:
# caminho modelo e preprocessador
pipeline_path = "../model/pipeline_predicao_default.pkl"

def prever_default(data: dict) -> dict:
    '''
    Função para prever se um cliente irá dar default ou não
    param data: dict com dados do cliente
    return: dict com previsão de default (0 ou 1)

    '''
    colunas_esperadas = [
        "default_3months", "ioi_36months", "ioi_3months", 
        "valor_por_vencer", "valor_vencido", "valor_quitado", 
        "quant_protestos", "valor_protestos", "quant_acao_judicial", 
        "acao_judicial_valor", "dividas_vencidas_valor", "dividas_vencidas_qtd", 
        "falencia_concordata_qtd", "tipo_sociedade", "opcao_tributaria",
        "atividade_principal", "forma_pagamento", "valor_total_pedido", 
        "month"
    ]
    
    # transforma dicionario em DataFrame
    df = pd.DataFrame([data])
    
    # remoção de colunas extras que não fazem parte do modelo 
    colunas_extras = set(df.columns) - set(colunas_esperadas)
    df = df.drop(columns=colunas_extras, errors="ignore")

    # verifica se colunas obrigatórios para o modelo estão presentes
    for col in colunas_esperadas:
        if col not in df.columns:
            raise ValueError(f"Falta coluna obrigatória: {col}")

    # removendo linhas com valores faltantes ou negativos
    df = df.loc[(df["tipo_sociedade"] != "missing") & (df["atividade_principal"] != "missing")]
    df = df[df["valor_total_pedido"] >= 0]
    df = df[df["forma_pagamento"] != "sem_pagamento"]

    # carregando modelo e preprocessamento
    pipeline = joblib.load(pipeline_path) 

    # aplicando a transformação da pipeline e fazendo a previsão
    prediction = pipeline.predict(df)

    return {"default": int(prediction)}

Exemplo de uso:

In [26]:
# teste

teste = {
    "default_3months": 2, 
    "ioi_36months": 45,  
    "ioi_3months": 30,  
    "valor_por_vencer": 0,  
    "valor_vencido": 0, 
    "valor_quitado": 120000.00,  
    "quant_protestos": 0,  
    "valor_protestos": 0,  
    "quant_acao_judicial": 0,  
    "acao_judicial_valor": 0.00, 
    "dividas_vencidas_valor": 5000.00, 
    "dividas_vencidas_qtd": 2,  
    "falencia_concordata_qtd": 0, 
    "tipo_sociedade": "sociedade empresarial",  
    "opcao_tributaria": "simples nacional",  
    "atividade_principal": "comercio",  
    "forma_pagamento": "boleto 30x60",  
    "valor_total_pedido": 35000000.00,  
    "month": 7
}

result = prever_default(teste)

print(result)

{'default': 1}
